# Quiz 4

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Question 3

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате. 

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён. 

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

Выборки независимы 

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [2]:
n1 = 34
n2 = 16
p1 = 10/n1
p2 = 4/n2

Считаем z статистику

In [3]:
def proportions_diff_z(p1, n1, p2, n2):
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    #print(P)
    denominator  =  np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
    #print(P * (1 - P), (1. / n1 + 1. / n2))
    return (p1 - p2) / denominator

In [125]:
z_btrakers = round(proportions_diff_z(p1, n1, p2, n2),4)
z_btrakers

0.3241

In [5]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [126]:
1 - stats.norm.cdf(z_btrakers)

0.3729311634638355

Критерий

## Question 4

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

In [6]:
df = pd.read_csv('banknotes.txt', sep="\t")
df

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

    логистическая регрессия по признакам X1,X2,X3X_1,X_2,X_3X1​,X2​,X3​;

    логистическая регрессия по признакам X4,X5,X6X_4,X_5,X_6X4​,X5​,X6​.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).

In [137]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('real', axis=1), df['real'], test_size=50, random_state=1)

In [138]:
y_test = y_test.to_numpy()
y_train = y_train.to_numpy()

In [139]:
len(X_train)

150

In [156]:
clf_1 = LogisticRegression().fit(X_train[['X1', 'X2', 'X3']], y_train)
pred_1 = clf_1.predict(X_test[['X1', 'X2', 'X3']])

In [157]:
pred_1

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0], dtype=int64)

In [158]:
clf_2 = LogisticRegression().fit(X_train[['X4', 'X5', 'X6']], y_train)
pred_2 = clf_2.predict(X_test[['X4', 'X5', 'X6']])

In [159]:
pred_2

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0], dtype=int64)

In [150]:
len(y_test)

50

In [151]:
#error_1 = (len(y_test) - np.sum(pred_1 == y_test))/len(y_test)
#error_2 = (len(y_test) - np.sum(pred_2 == y_test))/len(y_test)
#error_1, error_2

In [160]:
error_1 = np.array([1 if pred_1[i]==y_test[i] else 0 for i in range(len(y_test))])
error_2 = np.array([1 if pred_2[i]==y_test[i] else 0 for i in range(len(y_test))])

In [161]:
error_1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1])

In [162]:
error_2

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

Связанные выборки

In [77]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [179]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample1)
        
    f = sum([1 if (sample1[i] == 1 and sample2[i] == 0) else 0 for i in range(n)])
    g = sum([1 if (sample1[i] == 0 and sample2[i] == 1) else 0 for i in range(n)])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [180]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample1)
    
    f = sum([1 if (sample1[i] == 1 and sample2[i] == 0) else 0 for i in range(n)])
    g = sum([1 if (sample1[i] == 0 and sample2[i] == 1) else 0 for i in range(n)])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [176]:
#sample = zip(error_1, error_2)

In [181]:
#sample[0]

In [182]:
print("95%% confidence interval for a difference between proportions: [%.4f, %.4f]" \
      % proportions_diff_confint_rel(error_1, error_2))

95% confidence interval for a difference between proportions: [-0.2865, -0.0735]


-0.0599
95% confidence interval for a difference between predictions: [-0.3001, -0.0599]

In [183]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(error_1, error_2)))

p-value: 0.000923


In [184]:
print('p-value: %f' % proportions_diff_z_test(proportions_diff_z_stat_rel(error_2, error_1)))

p-value: 0.000923


In [185]:
proportions_diff_z_stat_rel(error_1, error_2)

-3.312945782245396

## Question 6

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов. 

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [115]:
n = 100
true_mean = 525
true_std = 100

x_mean = 541.4

интервальная оценка долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [116]:
def z_test(true_mean, x_mean, true_std, n):
    return (x_mean - true_mean)/(true_std/np.sqrt(n))

In [117]:
def p_val_greater(z_stat):
    return 1 - stats.norm.cdf(z_stat)

In [187]:
z = z_test(true_mean, x_mean, true_std, n)
z

1.6399999999999977

In [188]:
round(p_val_greater(z),4)

0.0505

## Question 7
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [189]:
new_mean = 541.5

In [190]:
z = z_test(true_mean, new_mean, true_std, n)
z

1.65

In [191]:
round(p_val_greater(z),4)

0.0495